In [1]:
import papersys
from papersys.database.manager import PaperManager
from papersys.database.name import *
from papersys.database.schema import PAPER_METADATA_SCHEMA
from papersys.database.migrate import load_arxiv_oai_snapshot, fetch_arxiv_oai
from papersys.const import DATA_DIR
from datetime import date


In [4]:
manager = PaperManager(DATA_DIR / "lancedb")

from pyarrow.parquet import write_table
write_table(
    manager.embedding_table.search().to_arrow(),
    DATA_DIR / "arxiv_embeddings.parquet"
)

2025-11-01 16:33:21.536 | INFO     | papersys.database.manager:__init__:28 - Connecting to database at /home/lyk/code/papersys/data/lancedb


In [5]:
import polars as pl

df = pl.read_parquet(DATA_DIR / "arxiv_embeddings.parquet")
df


id,embedding
str,"array[f32, 1536]"
"""2510.04553""","[-0.025192, 0.021515, … 0.008606]"
"""2510.06935""","[0.002327, -0.009773, … 0.01368]"
"""2510.00282""","[0.007599, -0.001456, … 0.019974]"
"""2510.06655""","[-0.00782, 0.010681, … 0.007298]"
"""2507.08704""","[-0.013557, 0.016785, … 0.015053]"
…,…
"""2510.24000""","[0.001866, 0.001362, … 0.015144]"
"""2510.24657""","[-0.016998, 0.00163, … 0.008072]"
"""2510.23989""","[-0.004524, -0.002932, … 0.003893]"


In [7]:
import pyarrow.parquet as pq

pq.write_table(
    df.to_arrow(),
    "embedding.parquet",
    compression="zstd",
    compression_level=5,
    data_page_version="2.0",
    use_byte_stream_split=["embedding.list.element"],   # 针对叶子 float32
    use_dictionary={"embedding.list.element": False},   # 关闭该叶子字典
)